In [47]:
import numpy as np # import for math and array operations
import pandas as pd # import for dataframe handle
import matplotlib.pyplot as plt # import for visual representation
import seaborn as sns # import for visual representation
import math
from bs4 import BeautifulSoup
import requests
import datetime as dt

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
df = pd.read_csv('../raw_data/steam_games.csv')

# Cleaning columns

### URL

### Types

In [49]:
new_df = df.copy().dropna(subset=['types'])
app_df = new_df[new_df['types'] == 'app']

### Names

### Desc_snippet

In [50]:
# Complete desc_snippet with game_description
for index in app_df['desc_snippet'].index:
    app_df.loc[index, 'desc_snippet'] = app_df.loc[index, 'game_description']

### Reviews

In [76]:
def clean_review(review):
    if '%' in str(review):
        return review.split(',')[0]
    else:
        return float('nan')

### Dates

In [52]:
def convert_date(date):
    try:
        return dt.datetime.strptime(date, "%b %d, %Y")
    except:
        return date
    
def valid_date(date):
    if isinstance(date, dt.datetime):
        return True
    else:
        return False

In [53]:
app_df['release_date'] = app_df['release_date'].apply(convert_date)

/var/folders/dv/7t_6tkbx07s6mfq_r5jxkp240000gp/T/ipykernel_29419/3659942068.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app_df['release_date'] = app_df['release_date'].apply(convert_date)


In [54]:
# add a validate column
app_df['valid_date'] = app_df['release_date'].apply(valid_date)

/var/folders/dv/7t_6tkbx07s6mfq_r5jxkp240000gp/T/ipykernel_29419/1590340263.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app_df['valid_date'] = app_df['release_date'].apply(valid_date)


### Developer

### Publisher

In [55]:
### Publisher cleaning sub
def clean_pub(x):
    if ',' in str(x):
        if x.split(',')[0] == x.split(',')[1]:
            return x.split(',')[0] 

app_df['publisher'] = app_df['publisher'].apply(lambda x : clean_pub(x))

/var/folders/dv/7t_6tkbx07s6mfq_r5jxkp240000gp/T/ipykernel_29419/3629656861.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app_df['publisher'] = app_df['publisher'].apply(lambda x : clean_pub(x))


In [56]:
#### Replacing missing publisher by developer - add inplace = True
app_df.publisher = app_df.publisher.fillna(app_df.developer)

/var/folders/dv/7t_6tkbx07s6mfq_r5jxkp240000gp/T/ipykernel_29419/1628064062.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app_df.publisher = app_df.publisher.fillna(app_df.developer)


### Popular tags

In [57]:
### creating a loop of all the different tags, calculating the occurences to select the top used.
tags = {}
for index, row in app_df.iterrows():
    tags_list= str(row['popular_tags']).split(',')
    for tag in tags_list:
        if not tag in tags:
            tags[tag] = 1
        else:
            tags[tag] += 1
            
            
tags_df = pd.DataFrame(list(tags.items()),columns = ['tag','count'])
tags_df.sort_values('count', ascending =False)

,tag,count
50,Indie,23718
2,Action,16112
42,Adventure,13378
75,Casual,12969
31,Simulation,9273
...,...,...
369,Snowboarding,3
364,BMX,3
362,ATV,3
202,Asymmetric VR,3


### Languages

In [58]:
### creating a loop of all the different languages, calculating the occurences to select the top used.
languages = {}
for index, row in app_df.iterrows():
    languages_list= str(row['languages']).split(',')
    for language in languages_list:
        if not language in languages:
            languages[language] = 1
        else:
            languages[language] += 1

languages_df = pd.DataFrame(list(languages.items()),columns = ['language','count'])
languages_df.sort_values('count', ascending =False)

,language,count
0,English,38007
3,German,13050
1,French,12512
4,Spanish - Spain,11554
8,Russian,9716
2,Italian,9620
5,Japanese,8693
11,Simplified Chinese,7116
7,Portuguese - Brazil,5326
6,Polish,4935


### Game details

In [59]:
### creating a loop of all the different details, calculating the occurences to select the top used.
details = {}
# Stripping game details
df.game_details = app_df.game_details.str.strip()
for index, row in app_df.iterrows():
    details_list= str(row['game_details']).split(',')
    for detail in details_list:
        if not detail in details:
            details[detail] = 1
        else:
            details[detail] += 1

details_df = pd.DataFrame(list(details.items()),columns = ['game_details','count'])
details_df.sort_values('count', ascending =False)

,game_details,count
0,Single-player,33797
3,Steam Achievements,19891
29,Downloadable Content,13169
6,Steam Cloud,11770
4,Steam Trading Cards,11587
14,Full controller support,8975
24,Profile Features Limited \r\n\t\t\t\t\t\t\t\t\t,8443
1,Multi-player,8386
5,Partial Controller Support,7081
17,Steam Leaderboards,6142


### Achievements

In [60]:
#categorization - adding a column for achievement cat
for i in app_df.index:
    app_df.at[i, 'achievement_cat'] = 0 if math.isnan(app_df.loc[i]['achievements']) == True else 1

/var/folders/dv/7t_6tkbx07s6mfq_r5jxkp240000gp/T/ipykernel_29419/1847830268.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app_df.at[i, 'achievement_cat'] = 0 if math.isnan(app_df.loc[i]['achievements']) == True else 1


### Genre

In [61]:
genre_dict = {}
for index, row in app_df.iterrows():
    genre_list= str(row['genre']).split(',')
    for genre in genre_list:
        if not genre in genre_dict:
            genre_dict[genre] = 1
        else:
            genre_dict[genre] += 1
genre_df = pd.DataFrame(list(genre_dict.items()),columns = ['genre','count'])
genre_df.sort_values('count', ascending =False) 

,genre,count
6,Indie,22868
0,Action,15265
10,Casual,12010
1,Adventure,11871
8,Simulation,8711
3,Strategy,7975
5,RPG,6962
7,Early Access,2818
4,Free to Play,2620
11,Sports,1725


### Game Desc

In [62]:
app_df['game_description']=app_df['game_description'].astype(str)

/var/folders/dv/7t_6tkbx07s6mfq_r5jxkp240000gp/T/ipykernel_29419/1020820551.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app_df['game_description']=app_df['game_description'].astype(str)


In [63]:
app_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38021 entries, 0 to 40832
Data columns (total 22 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   url                       38021 non-null  object 
 1   types                     38021 non-null  object 
 2   name                      38007 non-null  object 
 3   desc_snippet              37920 non-null  object 
 4   recent_reviews            2706 non-null   object 
 5   all_reviews               28470 non-null  object 
 6   release_date              37654 non-null  object 
 7   developer                 37721 non-null  object 
 8   publisher                 37779 non-null  object 
 9   popular_tags              37888 non-null  object 
 10  game_details              37546 non-null  object 
 11  languages                 38007 non-null  object 
 12  achievements              12194 non-null  float64
 13  genre                     37625 non-null  object 
 14  game_d

### Mature Content

In [64]:
app_df=app_df.assign(mature_encoded=lambda x: x.mature_content.isna().replace((True,False),(0,1)))

### Prices

In [65]:
app_df=app_df.assign(price=lambda x: pd.to_numeric(x.original_price.str.strip('$'), errors='coerce').replace(np.nan, 0))

# Cleaning columns

In [74]:
clean_df = app_df.drop(['mature_content', 'achievements', 'recent_reviews', 'desc_snippet', 'original_price', 'discount_price'], axis = 1)

In [75]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38021 entries, 0 to 40832
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   url                       38021 non-null  object 
 1   types                     38021 non-null  object 
 2   name                      38017 non-null  object 
 3   all_reviews               28470 non-null  object 
 4   release_date              37654 non-null  object 
 5   developer                 37721 non-null  object 
 6   publisher                 37779 non-null  object 
 7   popular_tags              37888 non-null  object 
 8   game_details              37546 non-null  object 
 9   languages                 38007 non-null  object 
 10  genre                     37625 non-null  object 
 11  game_description          38021 non-null  object 
 12  minimum_requirements      21069 non-null  object 
 13  recommended_requirements  21075 non-null  object 
 14  valid_

# Scraping Columns

### Names

In [70]:
def get_name(url):
    response = requests.get(url).text
    soup = BeautifulSoup(response, "html.parser")
    try: 
        return soup.find('h2', class_='pageheader').text.strip()
    except AttributeError:
        try:
            return soup.find('div', class_='apphub_AppName').text.strip()
        except AttributeError:
            return float('nan')

In [71]:
for index in clean_df[clean_df['name'].isnull()].index:
    clean_df.loc[index, 'name'] = get_name(clean_df.loc[index, 'url'])

In [73]:
clean_df[clean_df['name'].isnull()== True]

,url,types,name,desc_snippet,recent_reviews,all_reviews,release_date,developer,publisher,popular_tags,...,game_description,mature_content,minimum_requirements,recommended_requirements,original_price,discount_price,valid_date,achievement_cat,mature_encoded,price
6381,https://store.steampowered.com/bundle/7125/Fac...,app,NaN,About this bundle This bundle contains thre...,NaN,NaN,NaN,NaN,NaN,NaN,...,About this bundle This bundle contains thre...,NaN,NaN,NaN,NaN,$2.67,False,0.0,0,0.0
12146,https://store.steampowered.com/bundle/10912/Th...,app,NaN,About this bundle All Joe Papp content.,NaN,NaN,NaN,NaN,NaN,NaN,...,About this bundle All Joe Papp content.,NaN,NaN,NaN,NaN,$9.96,False,0.0,0,0.0
28380,https://store.steampowered.com/bundle/10417/Ha...,app,NaN,About this bundle Purchase this bundle to sa...,NaN,NaN,NaN,NaN,NaN,NaN,...,About this bundle Purchase this bundle to sa...,NaN,NaN,NaN,NaN,$16.55,False,0.0,0,0.0
31321,https://store.steampowered.com/app/976190/_/,app,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Action,Indie,Third-Person Shooter,Retro",...,nan,NaN,NaN,NaN,NaN,NaN,False,0.0,0,0.0


### Reviews

In [ ]:
def get_review(url):
    response = requests.get(url).text
    soup = BeautifulSoup(response, "html.parser")
    try: 
        return soup.find('span', class_='game_review_summary').text.strip()
    except AttributeError:
        return float('nan')

In [ ]:
for index in app_df[app_df['all_reviews'].isnull()].index:
    app_df.loc[index, 'all_reviews'] = get_review(app_df.loc[index, 'url'])

### Dates

In [12]:
def get_date(url):
    response = requests.get(url).text
    soup = BeautifulSoup(response, "html.parser")
    try: 
        return soup.find('div', class_='date').text.strip()
    except AttributeError:
        return float('nan')

In [ ]:
for index in app_df[app_df['release_date'].isnull()].index:
    app_df.loc[index, 'release_date'] = get_date(app_df.loc[index, 'url'])

### Developer

In [ ]:
def get_dev(url):
    response = requests.get(url).text
    soup = BeautifulSoup(response, "html.parser")
    try: 
        return soup.find('div', id='developers_list').text.strip().split(',')[0]
    except:
        return float('nan')

In [ ]:
temp = df_app[app_df['developer'].isnull() == True]
for index in temp.index:
    app_df.loc[index, 'developer'] = get_dev(app_df.loc[index, 'url'])
    print(index)

### Publisher

In [38]:
def get_pub(url):
    response = requests.get(url).text
    soup = BeautifulSoup(response, "html.parser")
    try: 
        return soup.find_all('div', class_='summary column')[3].find('a').string.strip().split(',')[0]
    except:
        return float('nan')

In [ ]:
temp = df_app[df_app['publisher'].isnull() == True]
for index in temp.index:
    df_app.loc[index, 'publisher'] = get_pub(df_app.loc[index, 'url'])

### Popular Tags

In [23]:
def get_tags(url):
    response = requests.get(url).text
    soup = BeautifulSoup(response, "html.parser")
    try: 
        return ','.join([tag.text.strip() for tag in soup.find_all('a', class_='app_tag')])
    except:
        return float('nan')

In [ ]:
temp = app_df[app_df['popular_tags'].isnull() == True]
for index in temp.index:
    app_df.loc[index, 'popular_tags'] = get_tags(app_df.loc[index, 'url'])
app_df['popular_tags'] = app_df['popular_tags'].replace('', float('nan'))

### Game Details

In [39]:
def get_det(url):
    response = requests.get(url).text
    soup = BeautifulSoup(response, "html.parser")
    try: 
        return ','.join([tag.text.strip() for tag in soup.find_all('div', class_='label')])
       # return soup.find_all('div', class_='label')
    except:
        return float('nan')

In [ ]:
temp = df_app[df_app['game_details'].isnull() == True]
for index in temp.index:
    app_df.loc[index, 'game_details'] = get_tags(app_df.loc[index, 'url'])
app_df['game_details'] = app_df['game_details'].replace('', float('nan'))

### Prices

In [40]:
def get_price(url):
    response = requests.get(url).text
    soup = BeautifulSoup(response, "html.parser")
    try: 
        return soup.find('div', class_='game_purchase_price.price').text.strip('€')
    except AttributeError:
        return float('nan')